In [46]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

In [47]:
import mlflow
import mlflow.pytorch

In [48]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///Users/ignacioarnesen/Documents/ITBA/Redes_Neuronales/skin-dataset-classification/mlruns/638978514444409565', creation_time=1750119976191, experiment_id='638978514444409565', last_update_time=1750119976191, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [49]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [50]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [51]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [52]:
# Crear directorio de logs
writer_base = SummaryWriter(log_dir="runs/mlp_base")
writer_dropout = SummaryWriter(log_dir="runs/mlp_dropout")
writer_batchnorm = SummaryWriter(log_dir="runs/mlp_batchnorm")
writer_dropout_batchnorm = SummaryWriter(log_dir="runs/mlp_dropout_batchnorm")

writer_L2_base = SummaryWriter(log_dir="runs/mlp_base_L2")
writer_L2_dropout = SummaryWriter(log_dir="runs/mlp_dropout_L2")
writer_L2_batchnorm = SummaryWriter(log_dir="runs/mlp_batchnorm_L2")
writer_L2_dropout_batchnorm = SummaryWriter(log_dir="runs/mlp_dropout_batchnorm_L2")

In [53]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [54]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(p=0.2, shift_limit=0.1, scale_limit=0.1, rotate_limit=90),
    A.Normalize(),
    ToTensorV2()
])


In [55]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [56]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [57]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [58]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

Vamos a hacer el clasificador con dropout

In [60]:
class MLPClassifierDropout(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.Dropout(0.5),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.Dropout(0.5),
        nn.ReLU(),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)

Ahora el que tiene batch normalizaztion

In [61]:
class MLPClassifierBatchNorm(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)

Ahora el que tiene dropout y batch normalization

In [62]:
class MLPClassifierDropoutBatchNorm(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model_base = MLPClassifier(num_classes=num_classes).to(device)
model_dropout = MLPClassifierDropout(num_classes=num_classes).to(device)
model_batchnorm = MLPClassifierBatchNorm(num_classes=num_classes).to(device)
model_dropout_batchnorm = MLPClassifierDropoutBatchNorm(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_base = optim.Adam(model_base.parameters(), lr=1e-3)
optimizer_dropout = optim.Adam(model_dropout.parameters(), lr=1e-3)
optimizer_batchnorm = optim.Adam(model_batchnorm.parameters(), lr=1e-3)
optimizer_dropout_batchnorm = optim.Adam(model_dropout_batchnorm.parameters(), lr=1e-3)

Ahora usemos un optimizer con Weight Decay

In [64]:
optimizer_L2_base = torch.optim.Adam(model_base.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_L2_dropout = torch.optim.Adam(model_dropout.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_L2_batchnorm = torch.optim.Adam(model_batchnorm.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_L2_dropout_batchnorm = torch.optim.Adam(model_dropout_batchnorm.parameters(), lr=0.001, weight_decay=1e-4)

In [65]:
# Entrenamiento y validación
def evaluate(model, loader, writer, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [44]:
# Loop de entrenamiento (modelo normal)
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model_base.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer_base.zero_grad()
            outputs = model_base(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_base.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model_base, val_loader, writer_base, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer_base.add_scalar("train/loss", train_loss, epoch)
        writer_base.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
    torch.save(model_base.state_dict(), "mlp_model_base.pth")
    print("Modelo guardado como 'mlp_model_base.pth'")
    mlflow.log_artifact("mlp_model_base.pth")
    mlflow.pytorch.log_model(model_base, artifact_path="pytorch_model")
    print("Modelo guardado como 'mlp_model_base.pth'")

Epoch 1/10: 100%|██████████| 22/22 [00:07<00:00,  2.89it/s]


Epoch 1:
  Train Loss: 3.0475, Accuracy: 28.41%
  Val   Loss: 1.9026, Accuracy: 40.33%


Epoch 2/10: 100%|██████████| 22/22 [00:07<00:00,  2.99it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Epoch 2:
  Train Loss: 1.7372, Accuracy: 43.33%
  Val   Loss: 1.4687, Accuracy: 50.83%


Epoch 3/10: 100%|██████████| 22/22 [00:06<00:00,  3.21it/s]


...

Epoch 9:
  Train Loss: 0.9792, Accuracy: 62.98%
  Val   Loss: 1.2307, Accuracy: 57.46%


Epoch 10/10: 100%|██████████| 22/22 [00:09<00:00,  2.30it/s]
2025/06/17 20:19:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 0.9251, Accuracy: 64.99%
  Val   Loss: 1.1747, Accuracy: 53.04%
Modelo guardado como 'mlp_model_base.pth'


2025/06/17 20:19:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_base.pth'


Ahora entrenamos el modelo con Dropout

In [45]:
# Loop de entrenamiento (modelo con dropout)
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifierDropout",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model_dropout.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer_dropout.zero_grad()
            outputs = model_dropout(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_dropout.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model_dropout, val_loader, writer_dropout, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer_dropout.add_scalar("train/loss", train_loss, epoch)
        writer_dropout.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
    torch.save(model_dropout.state_dict(), "mlp_model_dropout.pth")
    print("Modelo guardado como 'mlp_model_dropout.pth'")
    mlflow.log_artifact("mlp_model_dropout.pth")
    mlflow.pytorch.log_model(model_dropout, artifact_path="pytorch_model")
    print("Modelo guardado como 'mlp_model_dropout.pth'")

Epoch 1/10: 100%|██████████| 22/22 [00:09<00:00,  2.22it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Epoch 1:
  Train Loss: 3.8927, Accuracy: 18.94%
  Val   Loss: 2.1149, Accuracy: 30.94%


Epoch 2/10: 100%|██████████| 22/22 [00:09<00:00,  2.38it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Epoch 2:
  Train Loss: 2.5098, Accuracy: 22.53%
  Val   Loss: 2.1011, Accuracy: 20.44%


Epoch 3/10: 100%|██████████| 22/22 [00:08<00:00,  2.53it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

...

Epoch 9:
  Train Loss: 1.6638, Accuracy: 39.17%
  Val   Loss: 1.4890, Accuracy: 43.09%


Epoch 10/10: 100%|██████████| 22/22 [00:09<00:00,  2.43it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 

Epoch 10:
  Train Loss: 1.6233, Accuracy: 37.16%
  Val   Loss: 1.4083, Accuracy: 47.51%
Modelo guardado como 'mlp_model_dropout.pth'


2025/06/17 20:21:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_dropout.pth'


Ahora entrenamos con Batch Normalization

In [46]:
# Loop de entrenamiento (modelo con BatchNorm)
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifierBatchNorm",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model_batchnorm.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer_batchnorm.zero_grad()
            outputs = model_batchnorm(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_batchnorm.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model_batchnorm, val_loader, writer_batchnorm, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer_batchnorm.add_scalar("train/loss", train_loss, epoch)
        writer_batchnorm.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
    torch.save(model_batchnorm.state_dict(), "mlp_model_batchnorm.pth")
    print("Modelo guardado como 'mlp_model_batchnorm.pth'")
    mlflow.log_artifact("mlp_model_batchnorm.pth")
    mlflow.pytorch.log_model(model_batchnorm, artifact_path="pytorch_model")
    print("Modelo guardado como 'mlp_model_batchnorm.pth'")

Epoch 1/10: 100%|██████████| 22/22 [00:09<00:00,  2.38it/s]


Epoch 1:
  Train Loss: 1.7776, Accuracy: 35.72%
  Val   Loss: 1.5377, Accuracy: 41.99%


Epoch 2/10: 100%|██████████| 22/22 [00:09<00:00,  2.34it/s]


Epoch 2:
  Train Loss: 1.4627, Accuracy: 46.48%
  Val   Loss: 1.4369, Accuracy: 45.30%


Epoch 3/10: 100%|██████████| 22/22 [00:09<00:00,  2.30it/s]


...

Epoch 9:
  Train Loss: 0.8642, Accuracy: 70.73%
  Val   Loss: 1.2585, Accuracy: 50.83%


Epoch 10/10: 100%|██████████| 22/22 [00:08<00:00,  2.62it/s]
2025/06/17 20:23:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 0.8180, Accuracy: 69.15%
  Val   Loss: 1.0503, Accuracy: 60.77%
Modelo guardado como 'mlp_model_batchnorm.pth'


2025/06/17 20:23:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_batchnorm.pth'


Ahora entrenamos con Dropout y Batch Normalization

In [47]:
# Loop de entrenamiento (modelo con dropout y batch normalization)
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifierDropoutBatchNorm",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model_dropout_batchnorm.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer_dropout_batchnorm.zero_grad()
            outputs = model_dropout_batchnorm(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_dropout_batchnorm.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model_dropout_batchnorm, val_loader, writer_dropout_batchnorm, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer_dropout_batchnorm.add_scalar("train/loss", train_loss, epoch)
        writer_dropout_batchnorm.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
    torch.save(model_dropout_batchnorm.state_dict(), "mlp_model_dropout_batchnorm.pth")
    print("Modelo guardado como 'mlp_model_batchnorm.pth'")
    mlflow.log_artifact("mlp_model_dropout_batchnorm.pth")
    mlflow.pytorch.log_model(model_dropout_batchnorm, artifact_path="pytorch_model")
    print("Modelo guardado como 'mlp_model_dropout_batchnorm.pth'")

Epoch 1/10:   0%|          | 0/22 [00:00<?, ?it/s]

Epoch 1/10: 100%|██████████| 22/22 [00:07<00:00,  2.82it/s]
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ignacioarnesen/miniconda3/envs/skin-tp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `

Epoch 1:
  Train Loss: 1.9956, Accuracy: 29.12%
  Val   Loss: 1.6897, Accuracy: 36.46%


Epoch 2/10: 100%|██████████| 22/22 [00:07<00:00,  2.81it/s]


Epoch 2:
  Train Loss: 1.7332, Accuracy: 38.45%
  Val   Loss: 1.5905, Accuracy: 40.88%


...

Epoch 9:
  Train Loss: 1.2221, Accuracy: 54.09%
  Val   Loss: 1.1495, Accuracy: 56.91%


Epoch 10/10: 100%|██████████| 22/22 [00:08<00:00,  2.54it/s]
2025/06/17 20:26:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 1.1707, Accuracy: 55.24%
  Val   Loss: 1.2165, Accuracy: 50.28%
Modelo guardado como 'mlp_model_batchnorm.pth'


2025/06/17 20:26:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_dropout_batchnorm.pth'


Ahora vamos a hacer los 4 modelos con optimizador L2

In [48]:
# Loop de entrenamiento de los 4 modelos con optimizador L2)

modelos = [model_base, model_dropout, model_batchnorm, model_dropout_batchnorm]
writers_L2 = [writer_L2_base, writer_L2_dropout, writer_L2_batchnorm, writer_L2_dropout_batchnorm]
models_names = ["MLPClassifier", "MLPClassifierDropout", "MLPClassifierBatchNorm", "MLPClassifierDropoutBatchNorm"]
pth_paths = ["mlp_model_base_L2.pth", "mlp_model_dropout_L2.pth", "mlp_model_batchnorm_L2.pth", "mlp_model_dropout_batchnorm_L2.pth"]
optimizers_L2 = [optimizer_L2_base, optimizer_L2_dropout, optimizer_L2_batchnorm, optimizer_L2_dropout_batchnorm]

for model, writer, model_name, pth_path, optimizer in zip(modelos, writers_L2, models_names, pth_paths, optimizers_L2):
    print(f"Entrenando modelo: {model_name}")
    n_epochs = 10
    with mlflow.start_run():
        # Log hiperparámetros
        mlflow.log_params({
            "model": model_name,
            "input_size": 64*64*3,
            "batch_size": batch_size,
            "lr": 1e-3,
            "epochs": n_epochs,
            "optimizer": "Adam",
            "loss_fn": "CrossEntropyLoss",
            "train_dir": train_dir,
            "val_dir": val_dir,
        })
        for epoch in range(n_epochs):
            model.train()
            running_loss = 0.0
            correct, total = 0, 0
        
            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
                images, labels = images.to(device), labels.to(device)
        
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
            train_loss = running_loss / len(train_loader)
            train_acc = 100.0 * correct / total
            val_loss, val_acc = evaluate(model, val_loader, writer, epoch=epoch, prefix="val")
        
            print(f"Epoch {epoch+1}:")
            print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
        
            writer.add_scalar("train/loss", train_loss, epoch)
            writer.add_scalar("train/accuracy", train_acc, epoch)
        
            # Log en MLflow
            mlflow.log_metrics({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            }, step=epoch)
            # Guardar modelo
        torch.save(model.state_dict(), pth_path)
        print(f"Modelo guardado como '{pth_path}'")
        mlflow.log_artifact(pth_path)
        mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
        print(f"Modelo guardado como '{pth_path}'")

Entrenando modelo: MLPClassifier


Epoch 1/10: 100%|██████████| 22/22 [00:08<00:00,  2.65it/s]


Epoch 1:
  Train Loss: 0.9309, Accuracy: 62.98%
  Val   Loss: 1.1747, Accuracy: 53.04%


Epoch 2/10: 100%|██████████| 22/22 [00:08<00:00,  2.57it/s]


Epoch 2:
  Train Loss: 0.9227, Accuracy: 63.13%
  Val   Loss: 1.1747, Accuracy: 53.04%


...

Epoch 10:
  Train Loss: 1.1791, Accuracy: 54.52%
  Val   Loss: 1.1385, Accuracy: 56.91%


2025/06/17 20:35:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modelo guardado como 'mlp_model_dropout_batchnorm_L2.pth'


2025/06/17 20:36:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_dropout_batchnorm_L2.pth'


Ahora, re-entrenamos los 4 modelos con la nueva data_transform

In [63]:
writer_L2_base_DA = SummaryWriter(log_dir="runs/mlp_base_L2_DA")
writer_L2_dropout_DA = SummaryWriter(log_dir="runs/mlp_dropout_L2_DA")
writer_L2_batchnorm_DA = SummaryWriter(log_dir="runs/mlp_batchnorm_L2_DA")
writer_L2_dropout_batchnorm_DA = SummaryWriter(log_dir="runs/mlp_dropout_batchnorm_L2_DA")

modelos = [model_base, model_dropout, model_batchnorm, model_dropout_batchnorm]
writers_L2_DA = [writer_L2_base_DA, writer_L2_dropout_DA, writer_L2_batchnorm_DA, writer_L2_dropout_batchnorm_DA]
models_names = ["MLPClassifier", "MLPClassifierDropout", "MLPClassifierBatchNorm", "MLPClassifierDropoutBatchNorm"]
pth_paths = ["mlp_model_base_L2_DA.pth", "mlp_model_dropout_L2_DA.pth", "mlp_model_batchnorm_L2_DA.pth", "mlp_model_dropout_batchnorm_L2_DA.pth"]
optimizers_L2 = [optimizer_L2_base, optimizer_L2_dropout, optimizer_L2_batchnorm, optimizer_L2_dropout_batchnorm]

for model, writer, model_name, pth_path, optimizer in zip(modelos, writers_L2_DA, models_names, pth_paths, optimizers_L2):
    print(f"Entrenando modelo: {model_name}")
    n_epochs = 10
    with mlflow.start_run():
        # Log hiperparámetros
        mlflow.log_params({
            "model": model_name,
            "input_size": 64*64*3,
            "batch_size": batch_size,
            "lr": 1e-3,
            "epochs": n_epochs,
            "optimizer": "Adam",
            "loss_fn": "CrossEntropyLoss",
            "train_dir": train_dir,
            "val_dir": val_dir,
        })
        for epoch in range(n_epochs):
            model.train()
            running_loss = 0.0
            correct, total = 0, 0
        
            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
                images, labels = images.to(device), labels.to(device)
        
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
            train_loss = running_loss / len(train_loader)
            train_acc = 100.0 * correct / total
            val_loss, val_acc = evaluate(model, val_loader, writer, epoch=epoch, prefix="val")
        
            print(f"Epoch {epoch+1}:")
            print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
        
            writer.add_scalar("train/loss", train_loss, epoch)
            writer.add_scalar("train/accuracy", train_acc, epoch)
        
            # Log en MLflow
            mlflow.log_metrics({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            }, step=epoch)
            # Guardar modelo
        torch.save(model.state_dict(), pth_path)
        print(f"Modelo guardado como '{pth_path}'")
        mlflow.log_artifact(pth_path)
        mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
        print(f"Modelo guardado como '{pth_path}'")

Entrenando modelo: MLPClassifier


Epoch 1/10: 100%|██████████| 22/22 [00:07<00:00,  2.76it/s]


Epoch 1:
  Train Loss: 1.7692, Accuracy: 39.45%
  Val   Loss: 1.6302, Accuracy: 45.86%


Epoch 2/10: 100%|██████████| 22/22 [00:08<00:00,  2.72it/s]


Epoch 2:
  Train Loss: 1.5430, Accuracy: 45.77%
  Val   Loss: 1.5536, Accuracy: 44.20%


...

Epoch 9:
  Train Loss: 1.2962, Accuracy: 50.65%
  Val   Loss: 1.2074, Accuracy: 48.62%


Epoch 10/10: 100%|██████████| 22/22 [00:10<00:00,  2.19it/s]
2025/06/17 22:30:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 1.2491, Accuracy: 52.08%
  Val   Loss: 1.1744, Accuracy: 51.38%
Modelo guardado como 'mlp_model_dropout_batchnorm_L2_DA.pth'


2025/06/17 22:30:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model_dropout_batchnorm_L2_DA.pth'


In [ ]:
!tensorboard --logdir=runs

# Inicialización de Parámetros

Para hacer todo lo que dice a continuación vamos a utilizar el modelo que mejores resultados nos dieron: MLPClassifierDropoutBatchNorm con L2 y DA

In [70]:
class MLPClassifierXavier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)
                
class MLPClassifierHe(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
                
                
class MLPClassifierUniform(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_size, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, num_classes)
    )

    def forward(self, x):
        return self.model(x)
    
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight)
                nn.init.zeros_(m.bias)

In [71]:

modelo_xavier = MLPClassifierXavier(num_classes=num_classes).to(device)
modelo_he = MLPClassifierHe(num_classes=num_classes).to(device)
modelo_uniform = MLPClassifierUniform(num_classes=num_classes).to(device)

model_name_xavier = "MLPClassifierXavier"
model_name_he = "MLPClassifierHe"
model_name_uniform = "MLPClassifierUniform"

path_xavier = "mlp_model_xavier.pth"
path_he = "mlp_model_he.pth"
path_uniform = "mlp_model_uniform.pth"

optimizer_xavier = optim.Adam(modelo_xavier.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_he = optim.Adam(modelo_he.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_uniform = optim.Adam(modelo_uniform.parameters(), lr=0.001, weight_decay=1e-4)

writer_xavier = SummaryWriter(log_dir="runs/p_init/mlp_xavier")
writer_he = SummaryWriter(log_dir="runs/p_init/mlp_he")
writer_uniform = SummaryWriter(log_dir="runs/p_init/mlp_uniform")

In [72]:
modelos = [modelo_xavier, modelo_he, modelo_uniform]
writers = [writer_xavier, writer_he, writer_uniform]
models_names = ["MLPClassifierXavier", "MLPClassifierHe", "MLPClassifierUniform"]
pth_paths = ["path_xavier", "path_he", "path_uniform"]
optimizers = [optimizer_xavier, optimizer_he, optimizer_uniform]

for model, writer, model_name, pth_path, optimizer in zip(modelos, writers, models_names, pth_paths, optimizers):
    print(f"Entrenando modelo: {model_name}")
    n_epochs = 10
    with mlflow.start_run():
        # Log hiperparámetros
        mlflow.log_params({
            "model": model_name,
            "input_size": 64*64*3,
            "batch_size": batch_size,
            "lr": 1e-3,
            "epochs": n_epochs,
            "optimizer": "Adam",
            "loss_fn": "CrossEntropyLoss",
            "train_dir": train_dir,
            "val_dir": val_dir,
        })
        for epoch in range(n_epochs):
            model.train()
            running_loss = 0.0
            correct, total = 0, 0
        
            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
                images, labels = images.to(device), labels.to(device)
        
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
                running_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
            train_loss = running_loss / len(train_loader)
            train_acc = 100.0 * correct / total
            val_loss, val_acc = evaluate(model, val_loader, writer, epoch=epoch, prefix="val")
        
            print(f"Epoch {epoch+1}:")
            print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
            print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
        
            writer.add_scalar("train/loss", train_loss, epoch)
            writer.add_scalar("train/accuracy", train_acc, epoch)
            
            if epoch == 0:
                for name, param in model.named_parameters():
                    writer.add_histogram(name, param, epoch)
        
            # Log en MLflow
            mlflow.log_metrics({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc
            }, step=epoch)
            # Guardar modelo
        torch.save(model.state_dict(), pth_path)
        print(f"Modelo guardado como '{pth_path}'")
        mlflow.log_artifact(pth_path)
        mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
        print(f"Modelo guardado como '{pth_path}'")

Entrenando modelo: MLPClassifierXavier


Epoch 1/10: 100%|██████████| 22/22 [00:08<00:00,  2.57it/s]


Epoch 1:
  Train Loss: 1.9942, Accuracy: 30.70%
  Val   Loss: 1.7496, Accuracy: 36.46%


Epoch 2/10: 100%|██████████| 22/22 [00:08<00:00,  2.52it/s]


Epoch 2:
  Train Loss: 1.7366, Accuracy: 38.31%
  Val   Loss: 1.6412, Accuracy: 42.54%


...

Epoch 9:
  Train Loss: 1.2548, Accuracy: 51.94%
  Val   Loss: 1.1660, Accuracy: 54.14%


Epoch 10/10: 100%|██████████| 22/22 [00:08<00:00,  2.69it/s]
2025/06/18 22:57:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 1.2575, Accuracy: 52.37%
  Val   Loss: 1.1887, Accuracy: 54.70%
Modelo guardado como 'path_uniform'


2025/06/18 22:58:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'path_uniform'


In [36]:
!tensorboard --logdir=runs/p_init

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.19.0 at http://localhost:6006/ (Press CTRL+C to quit)
W0617 17:33:38.568465 123145593647104 application.py:559] path /apple-touch-icon-precomposed.png not found, sending 404
W0617 17:33:38.817898 123145593647104 application.py:559] path /apple-touch-icon.png not found, sending 404
^C
